In [1]:
import pandas as pd
import sys
import numpy as np
import os
import mysql.connector
from tqdm import tqdm

In [3]:
path_model = os.path.join(os.getcwd(),'DATABASE_DP')
path_dwh = os.path.join(os.getcwd(),'..','3_data_preprocesing_module','DATAWAREHOUSE')

In [7]:
filelist = os.listdir(path_model)
filelist

['.ipynb_checkpoints',
 'ci_cp_num_cap.csv',
 'ci_cp_total_ganancias.csv',
 'ci_cp_total_horas.csv',
 'ci_rs_total_cursos.csv',
 'ci_rs_volumen_cap.csv',
 'ci_rs_volumen_ganancias.csv',
 'ci_rs_volumen_horas.csv',
 'oc_cp_num_cap_cer.csv',
 'oc_cp_porcentaje_asistencia.csv',
 'oc_cp_total_ganancias.csv',
 'oc_cp_total_horas.csv',
 'oc_rs_total_cursos.csv',
 'oc_rs_volumen_cap_cer.csv',
 'oc_rs_volumen_ganancias.csv',
 'oc_rs_volumen_horas.csv',
 'oc_rs_volumen_porcentaje_asistencia.csv',
 'oec_cp_num_cer.csv',
 'oec_cp_porcentaje_asistencia.csv',
 'oec_cp_total_ganancias.csv',
 'oec_cp_total_horas.csv',
 'oec_rs_total_perfiles.csv',
 'oec_rs_volumen_cer.csv',
 'oec_rs_volumen_ganancias.csv',
 'oec_rs_volumen_horas.csv',
 'oec_rs_volumen_porcentaje_asistencia.csv']

In [95]:
df_aux = pd.read_csv(os.path.join(path_model,'oec_cp_total_ganancias.csv'))

In [97]:
df_aux.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18275 entries, 0 to 18274
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   area_familia                 18275 non-null  object 
 1   especialidad_sector          18275 non-null  object 
 2   estado                       18275 non-null  object 
 3   canton                       18275 non-null  object 
 4   provincia                    18275 non-null  object 
 5   id_fact_oec_cp               18275 non-null  float64
 6   id_razon_social              18275 non-null  float64
 7   id_curso_perfil              18275 non-null  float64
 8   id_fecha                     18275 non-null  float64
 9   porcentaje_asistencia        18275 non-null  float64
 10  carga_horaria                18275 non-null  float64
 11  costo                        18275 non-null  float64
 12  total_ganancias              18275 non-null  float64
 13  tipo            

In [93]:
df_aux2.tail()

,area_familia,fecha,provincia,id_fact_ci_cp,id_razon_social,id_curso_perfil,id_fecha,dia,mes,total_ganancias,...,estado,tipo_razon_social,fecha_resolucion,canton,num_cap,total_horas,carga_horaria,costo,anio,prediction(total_ganancias)
2204,tecnologías de la información y comunicación,31-12-2020,pichincha,1102.0,2210.0,8811.0,5.0,31.0,12.0,91804.80,...,calificado,capacitador independiente,resolución no. mdt-ci-cal-2020-0477,quito,438.0,17520.0,40.0,209.60,2020.0,91804.80
2206,construcción e infraestructura,31-12-2020,azuay,1103.0,2213.0,8837.0,5.0,31.0,12.0,3306.72,...,calificado,capacitador independiente,resolución no. mdt-ci-cal-2020-0494,cuenca,166.0,3984.0,24.0,19.92,2020.0,3306.72
2208,procesos industriales,31-12-2020,guayas,1104.0,2217.0,8788.0,5.0,31.0,12.0,1449.60,...,calificado,capacitador independiente,resolución no. mdt-ci-cal-2020-0502,guayaquil,5.0,80.0,16.0,289.92,2020.0,1449.60
2210,procesos industriales,31-12-2020,pichincha,1105.0,2224.0,8812.0,5.0,31.0,12.0,6567.60,...,calificado,capacitador independiente,resolución no. mdt-ci-cal-2020-0481,quito,39.0,1560.0,40.0,168.40,2020.0,6567.60
2212,procesos industriales,31-12-2020,pichincha,1106.0,2232.0,8845.0,5.0,31.0,12.0,4885.92,...,calificado,capacitador independiente,resolución no. mdt-ci-cal-2020-0493,quito,29.0,696.0,24.0,168.48,2020.0,4885.92


In [91]:
df_aux['prediction(total_ganancias)'] = df_aux2['prediction(total_ganancias)']

In [92]:
df_aux[['prediction(num_cap)','prediction(total_ganancias)']].tail()

,prediction(num_cap),prediction(total_ganancias)
2204,438.0,91804.80
2206,166.0,3306.72
2208,5.0,1449.60
2210,39.0,6567.60
2212,29.0,4885.92


In [54]:
df = []
fact_oc_cp = None
fact_oec_cp = None
fact_ci_cp = None
fact_oc_rs = None
fact_oec_rs = None
fact_ci_rs = None
for file in filelist:
    if '.csv' in file:
            
        df = pd.read_csv(os.path.join(path_model,file))
        if 'oc_cp_' in file:
            if fact_oc_cp is None:
                fact_oc_cp = df
            else: 
                fact_oc_cp = fact_oc_cp.merge(df,on='id_fact_oc_cp',how='inner')
                print(fact_oc_cp.shape, df.shape)
        elif 'oec_cp_' in file:
            if fact_oec_cp is None:
                fact_oec_cp = df
            else:
                fact_oec_cp = fact_oec_cp.merge(df,on='id_fact_oec_cp',how='inner')
                print(fact_oec_cp.shape, df.shape)
        elif 'ci_cp_' in file:
            if fact_ci_cp is None:
                fact_ci_cp = df
            else:
                fact_ci_cp = fact_ci_cp.merge(df,on='id_fact_ci_cp',how='inner')
                print(fact_ci_cp.shape, df.shape)
        elif 'oc_rs_' in file:
            if fact_oc_rs is None:
                fact_oc_rs = df
            else:
                fact_oc_rs = fact_oc_rs.merge(df,on='id_fact_oc_rs',how='inner')
                print(fact_oc_rs.shape, df.shape)
        elif 'oec_rs_' in file:
            if fact_oec_rs is None:
                fact_oec_rs = df
            else:
                fact_oec_rs = fact_oec_rs.merge(df,on='id_fact_oec_rs',how='inner')
                print(fact_oec_rs.shape, df.shape)
        elif 'ci_rs_' in file:
            if fact_ci_rs is None:
                fact_ci_rs = df
            else:
                fact_ci_rs = fact_ci_rs.merge(df,on='id_fact_ci_rs',how='inner')
                print(fact_ci_rs.shape, df.shape)

(4428, 49) (2214, 25)
(8856, 73) (2214, 25)
(2508, 35) (1254, 18)
(5016, 52) (1254, 18)
(10032, 69) (1254, 18)


C:\Users\sjime\AppData\Local\Temp\ipykernel_29856\470083393.py:46: FutureWarning: Passing 'suffixes' which cause duplicate columns {'mes_x', 'tipo_razon_social_x', 'volumen_cap_x', 'fecha_resolucion_x', 'total_cursos_x', 'id_razon_social_x', 'id_fecha_x', 'anio_x', 'dia_x', 'volumen_ganancias_x', 'provincia_x', 'razon_social_x', 'volumen_horas_x', 'canton_x', 'fecha_x', 'estado_x'} in the result is deprecated and will raise a MergeError in a future version.
  fact_ci_rs = fact_ci_rs.merge(df,on='id_fact_ci_rs',how='inner')


(7728, 51) (3864, 26)
(15456, 76) (3864, 26)
(30912, 101) (3864, 26)


C:\Users\sjime\AppData\Local\Temp\ipykernel_29856\470083393.py:16: FutureWarning: Passing 'suffixes' which cause duplicate columns {'mes_x', 'tipo_razon_social_x', 'total_ganancias_x', 'id_fecha_x', 'razon_social_x', 'area_familia_x', 'fecha_x', 'carga_horaria_x', 'costo_x', 'anio_x', 'porcentaje_asistencia_x', 'fecha_resolucion_x', 'num_cap_cer_x', 'id_curso_perfil_x', 'id_razon_social_x', 'total_horas_x', 'dia_x', 'provincia_x', 'canton_x', 'estado_x', 'modalidad_x', 'tipo_x', 'curso_perfil_x', 'especialidad_sector_x'} in the result is deprecated and will raise a MergeError in a future version.
  fact_oc_cp = fact_oc_cp.merge(df,on='id_fact_oc_cp',how='inner')
C:\Users\sjime\AppData\Local\Temp\ipykernel_29856\470083393.py:34: FutureWarning: Passing 'suffixes' which cause duplicate columns {'mes_x', 'tipo_razon_social_x', 'fecha_resolucion_x', 'total_cursos_x', 'id_razon_social_x', 'id_fecha_x', 'volumen_cap_cer_x', 'dia_x', 'volumen_ganancias_x', 'provincia_x', 'razon_social_x', 'vol

(1132, 37) (566, 19)
(2264, 55) (566, 19)
(4528, 73) (566, 19)
(9056, 91) (566, 19)
(18403, 51) (18275, 26)
(18659, 76) (18275, 26)


C:\Users\sjime\AppData\Local\Temp\ipykernel_29856\470083393.py:22: FutureWarning: Passing 'suffixes' which cause duplicate columns {'mes_x', 'tipo_razon_social_x', 'total_ganancias_x', 'id_fecha_x', 'razon_social_x', 'area_familia_x', 'fecha_x', 'carga_horaria_x', 'costo_x', 'anio_x', 'porcentaje_asistencia_x', 'fecha_resolucion_x', 'id_curso_perfil_x', 'id_razon_social_x', 'total_horas_x', 'dia_x', 'provincia_x', 'canton_x', 'estado_x', 'modalidad_x', 'tipo_x', 'curso_perfil_x', 'especialidad_sector_x', 'num_cer_x'} in the result is deprecated and will raise a MergeError in a future version.
  fact_oec_cp = fact_oec_cp.merge(df,on='id_fact_oec_cp',how='inner')


(19171, 101) (18275, 26)
(12874, 37) (12778, 19)
(13066, 55) (12778, 19)
(13450, 73) (12778, 19)
(14218, 91) (12778, 19)


C:\Users\sjime\AppData\Local\Temp\ipykernel_29856\470083393.py:40: FutureWarning: Passing 'suffixes' which cause duplicate columns {'mes_x', 'tipo_razon_social_x', 'volumen_cer_x', 'fecha_resolucion_x', 'id_razon_social_x', 'id_fecha_x', 'anio_x', 'volumen_ganancias_x', 'razon_social_x', 'provincia_x', 'volumen_horas_x', 'volumen_porcentaje_asistencia_x', 'dia_x', 'canton_x', 'total_perfiles_x', 'fecha_x', 'estado_x'} in the result is deprecated and will raise a MergeError in a future version.
  fact_oec_rs = fact_oec_rs.merge(df,on='id_fact_oec_rs',how='inner')


In [48]:
fact_oc_cp = fact_oc_cp[['id_fact_oc_cp','id_razon_social','id_curso_perfil','num_cap_cer','prediction(num_cap_cer)','total_horas','prediction(total_horas)', 'total_ganancias', 'prediction(total_ganancias)', 'porcentaje_asistencia','prediction(porcentaje_asistencia)']]

In [52]:
fact_oc_cp.shape

(30912, 11)

In [111]:
df = pd.read_csv(os.path.join(path_model,'data_oc_kpi6.csv'))
df.head()

,area_familia,especialidad_sector,modalidad,razon_social,fecha_resolucion,canton,provincia,id_fact_oc,id_razon_social,id_curso_perfil,...,estado,tipo_razon_social,num_cap_cer,total_cursos,volumen_capacitados,total_horas,carga_horaria,anio,%_asistencia,prediction(%_asistencia)
0,construcción e infraestructura,tecnología de la construcción,presencial,MISSING,MISSING,quito,pichincha,0.0,1.0,3487.0,...,calificado,operador de capacitación,34.0,4.0,52.0,1360.0,40.0,2021.0,0.0,6.100755
1,educación y capacitación,formación de instructores facilitadores monito...,presencial,MISSING,MISSING,quito,pichincha,1.0,1.0,3490.0,...,calificado,operador de capacitación,1.0,4.0,52.0,150.0,150.0,2021.0,0.0,6.100755
2,servicios socioculturales y a la comunidad,salud y medicina,presencial,MISSING,MISSING,MISSING,MISSING,2.0,4.0,2868.0,...,calificado,operador de capacitación,25.0,12.0,500.0,1500.0,60.0,2021.0,0.0,6.100755
3,servicios socioculturales y a la comunidad,MISSING,presencial,MISSING,MISSING,guayaquil,guayas,3.0,12.0,3944.0,...,calificado,operador de capacitación,5.0,2.0,249.0,150.0,30.0,2021.0,0.0,6.100755
4,servicios socioculturales y a la comunidad,MISSING,presencial,MISSING,MISSING,guayaquil,guayas,4.0,12.0,3943.0,...,calificado,operador de capacitación,244.0,2.0,249.0,9760.0,40.0,2021.0,0.0,77.503424


# Conexión con MySQL

In [7]:
mydb = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='KappaTao1233'
)

mycursor = mydb.cursor()

# Creación de la base de datos¶

In [55]:
mycursor.execute("DROP DATABASE IF EXISTS dwh_setec")
mycursor.execute("CREATE DATABASE dwh_setec")
mycursor.execute("USE dwh_setec")

# DIMENSIÓN RAZÓN SOCIAL

In [56]:
df_razon_social = pd.read_csv(os.path.join(path_dwh,'dim_razon_social.csv'))

In [57]:
df_razon_social = df_razon_social[['id_razon_social', 'razon_social', 'estado', 'tipo_razon_social', 'canton', 'provincia']]

In [58]:
mycursor.execute("DROP TABLE IF EXISTS dim_razon_social")
mycursor.execute('''CREATE TABLE dim_razon_social (
id_razon_social INT PRIMARY KEY,
razon_social VARCHAR(255) NOT NULL,
tipo_razon_social VARCHAR(255) NOT NULL,
estado VARCHAR(63) NOT NULL,
canton VARCHAR(63) NOT NULL,
provincia VARCHAR(63) NOT NULL
)''')
sql_formula = 'INSERT INTO dim_razon_social ( id_razon_social, razon_social, estado, tipo_razon_social, canton, provincia) VALUES (%s,%s,%s,%s,%s,%s)'
for row in tqdm(df_razon_social.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

100%|██████████████████████████████████████████████████████████████████████████████| 2257/2257 [01:36<00:00, 23.50it/s]


# DATAMART OC

## Tabla dim_curso_perfil_oc

In [59]:
df_cur_per = pd.read_csv(os.path.join(path_dm_oc,'dim_curso_perfil.csv'))
df_cur_per['id_curso_perfil'] = df_cur_per['id_curso_perfil'].astype(str)
df_cur_per = df_cur_per.fillna({'costo':0})

In [60]:
mycursor.execute("DROP TABLE IF EXISTS dim_curso_perfil_oc")
mycursor.execute('''CREATE TABLE dim_curso_perfil_oc (
id_curso_perfil INT PRIMARY KEY,
tipo VARCHAR(63) NOT NULL,
area_familia VARCHAR(255) NOT NULL,
especialidad_sector VARCHAR(255) NOT NULL,
curso_perfil VARCHAR(255) NOT NULL,
modalidad VARCHAR(255) NOT NULL,
carga_horaria FLOAT NOT NULL,
costo FLOAT NOT NULL
)''')
sql_formula = 'INSERT INTO dim_curso_perfil_oc ( id_curso_perfil, tipo, area_familia, especialidad_sector, curso_perfil, modalidad, carga_horaria, costo) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)'
for row in tqdm(df_cur_per.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

100%|██████████████████████████████████████████████████████████████████████████████| 5103/5103 [04:00<00:00, 21.26it/s]


## Tabla dim_fecha_oc

In [92]:
df_fecha_oc = pd.read_csv(os.path.join(path_dm_oc,'dim_fecha.csv'))
df_fecha_oc['id_fecha'] = df_fecha_oc['id_fecha'].astype(str)
df_fecha_oc['anio'] = df_fecha_oc['anio'].astype(str)

In [93]:
mycursor.execute("DROP TABLE IF EXISTS dim_fecha_oc")
mycursor.execute('''CREATE TABLE dim_fecha_oc (
id_fecha INT PRIMARY KEY,
anio VARCHAR(4) NOT NULL
)''')
sql_formula = 'INSERT INTO dim_fecha_oc ( id_fecha, anio) VALUES (%s,%s)'
for row in tqdm(df_fecha_oc.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 25.63it/s]


## Tabla fact_oc

In [94]:
df_fact_oc = fact_oc[['id_fact_oc','id_razon_social', 'id_curso_perfil', 'id_fecha', 'num_cap_cer', 'prediction(num_cap_cer)', 'total_cursos', 'prediction(total_cursos)', 'volumen_capacitados', 'prediction(volumen_capacitados)', 'total_horas', 'prediction(total_horas)', 'total_ganancias', 'prediction(total_ganancias)','%_asistencia','prediction(%_asistencia)']]
df_fact_oc.columns = ['id_fact_oc','id_razon_social', 'id_curso_perfil', 'id_fecha', 'num_cap_cer', 'prediction_num_cap_cer', 'total_cursos', 'prediction_total_cursos', 'volumen_capacitados', 'prediction_volumen_capacitados', 'total_horas', 'prediction_total_horas', 'total_ganancias', 'prediction_total_ganancias','%_asistencia','prediction_%_asistencia']
df_fact_oc['id_razon_social'] = df_fact_oc['id_razon_social'].astype(str)
df_fact_oc['id_curso_perfil'] = df_fact_oc['id_curso_perfil'].astype(str)
df_fact_oc['id_fecha'] = df_fact_oc['id_fecha'].astype(str)

C:\Users\sjime\AppData\Local\Temp\ipykernel_31844\470633657.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fact_oc['id_razon_social'] = df_fact_oc['id_razon_social'].astype(str)
C:\Users\sjime\AppData\Local\Temp\ipykernel_31844\470633657.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fact_oc['id_curso_perfil'] = df_fact_oc['id_curso_perfil'].astype(str)
C:\Users\sjime\AppData\Local\Temp\ipykernel_31844\470633657.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [108]:
df_fact_oc.isnull().sum()

id_fact_oc                           0
id_razon_social                      0
id_curso_perfil                      0
id_fecha                             0
num_cap_cer                          0
prediction_num_cap_cer               0
total_cursos                         0
prediction_total_cursos              0
volumen_capacitados                  0
prediction_volumen_capacitados       0
total_horas                          0
prediction_total_horas              36
total_ganancias                      0
prediction_total_ganancias        1432
%_asistencia                         0
prediction_%_asistencia           1238
dtype: int64

In [105]:
mycursor.execute("DROP TABLE IF EXISTS fact_oc")
mycursor.execute('''CREATE TABLE fact_oc (
id_fact_oc INT PRIMARY KEY,
id_razon_social INT NOT NULL,
id_curso_perfil INT NOT NULL,
id_fecha INT NOT NULL,
num_cap_cer INT NOT NULL,
prediction_num_cap_cer INT NOT NULL,
total_cursos INT NOT NULL,
prediction_total_cursos INT NOT NULL,
volumen_capacitados INT NOT NULL,
prediction_volumen_capacitados INT NOT NULL,
total_horas INT NOT NULL,
prediction_total_horas INT NOT NULL,
total_ganancias FLOAT NOT NULL,
prediction_total_ganancias FLOAT NOT NULL,
porcentaje_asistencia FLOAT NOT NULL,
prediction_porcentaje_asistencia FLOAT NOT NULL
)''')
sql_formula = '''INSERT INTO fact_oc (id_fact_oc,id_razon_social, id_curso_perfil, id_fecha, num_cap_cer, prediction_num_cap_cer, total_cursos, prediction_total_cursos, volumen_capacitados, prediction_volumen_capacitados, total_horas, prediction_total_horas, total_ganancias, prediction_total_ganancias,porcentaje_asistencia,prediction_porcentaje_asistencia) 
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
for row in tqdm(df_fact_oc.values):
    print(tuple(row))
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

  0%|                                                                                         | 0/1580 [00:00<?, ?it/s]

(0.0, '1.0', '3487.0', '6.0', 34.0, 34.0, 4.0, 4.666666666666667, 52.0, 113.975, 1360.0, 1390.0, 0.0, nan, 0.0, nan)


ProgrammingError: 1054 (42S22): Unknown column 'nan' in 'field list'

## CREAR CLAVES FORANEAS OC

In [54]:
mycursor.execute(''' 
ALTER TABLE fact_oc
ADD FOREIGN KEY (id_fecha) REFERENCES dim_fecha_oc(id_fecha)
''')
mycursor.execute(''' 
ALTER TABLE fact_oc
ADD FOREIGN KEY (id_razon_social) REFERENCES dim_razon_social(id_razon_social)
''')
mycursor.execute(''' 
ALTER TABLE fact_oc
ADD FOREIGN KEY (id_curso_perfil) REFERENCES dim_curso_perfil_oc(id_curso_perfil)
''')

DatabaseError: 3780 (HY000): Referencing column 'id_fecha' and referenced column 'id_fecha' in foreign key constraint 'fact_oc_ibfk_1' are incompatible.

# DATAMART OEC

In [ ]:
df_oec = pd.read_csv(os.path.join(path_dm_oec,'dim_razon_social_oec.csv'))
df_oec['id_oec'] = df_oec['id_oec'].astype(str)

In [ ]:
mycursor.execute("DROP TABLE IF EXISTS dim_razon_social_oec")
mycursor.execute('''CREATE TABLE dim_razon_social_oec (
id_oec VARCHAR(4) PRIMARY KEY,
razon_social VARCHAR(255) NOT NULL,
estado VARCHAR(63) NOT NULL,
canton VARCHAR(63) NOT NULL,
provincia VARCHAR(63) NOT NULL
)''')
sql_formula = 'INSERT INTO dim_razon_social_oec ( id_oec, razon_social, estado, canton, provincia) VALUES (%s,%s,%s,%s,%s)'
for row in tqdm(df_oec.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

## Tabla dim_perfil

In [ ]:
df_perfil = pd.read_csv(os.path.join(path_dm_oec,'dim_perfil.csv'))
df_perfil['id_perfil'] = df_perfil['id_perfil'].astype(str)

In [ ]:
mycursor.execute("DROP TABLE IF EXISTS dim_perfil_oec")
mycursor.execute('''CREATE TABLE dim_perfil_oec (
id_perfil VARCHAR(4) PRIMARY KEY,
familia VARCHAR(255) NOT NULL,
sector VARCHAR(255) NOT NULL,
perfil VARCHAR(255) NOT NULL,
modalidad VARCHAR(63) NOT NULL,
carga_horaria INT NOT NULL,
costo FLOAT NOT NULL
)''')
sql_formula = 'INSERT INTO dim_perfil_oec (id_perfil, familia, sector, perfil, modalidad, carga_horaria, costo) VALUES (%s,%s,%s,%s,%s,%s,%s)'
for row in tqdm(df_perfil.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

## Tabla dim_fecha_oec

In [ ]:
df_fecha_oec = pd.read_csv(os.path.join(path_dm_oec,'dim_fecha.csv'))
df_fecha_oec['id_fecha'] = df_fecha_oec['id_fecha'].astype(str)

In [ ]:
mycursor.execute("DROP TABLE IF EXISTS dim_fecha_oec")
mycursor.execute('''CREATE TABLE dim_fecha_oec (
id_fecha VARCHAR(4) PRIMARY KEY,
dia INT NOT NULL,
mes INT NOT NULL,
anio INT NOT NULL
)''')
sql_formula = 'INSERT INTO dim_fecha_oec (id_fecha, dia, mes, anio) VALUES (%s,%s,%s,%s)'
for row in tqdm(df_fecha_oec.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

## Tabla fact_oec

In [ ]:
df_fact_oec = fact_oec
df_fact_oec['id_razon_social'] = df_fact_oec['id_razon_social'].astype(str)
df_fact_oec['id_perfil'] = df_fact_oec['id_perfil'].astype(str)
df_fact_oec['id_fecha'] = df_fact_oec['id_fecha'].astype(str)

In [ ]:
mycursor.execute("DROP TABLE IF EXISTS fact_oec")
mycursor.execute('''CREATE TABLE fact_oec (
id_razon_social VARCHAR(4) NOT NULL,
id_perfil VARCHAR(4) NOT NULL,
id_fecha VARCHAR(4) NOT NULL,
num_cer INT NOT NULL,
total_horas INT NOT NULL,
total_cursos INT NOT NULL,
volumen_capacitados INT NOT NULL,
total_ganancias FLOAT NOT NULL
)''')
sql_formula = 'INSERT INTO fact_oec(id_razon_social, id_perfil, id_fecha, num_cer, total_horas, total_cursos, volumen_capacitados, total_ganancias) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)'
for row in tqdm(df_fact_oec.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

## CREAR CLAVES FORANEAS OEC

In [ ]:
mycursor.execute(''' 
ALTER TABLE fact_oec
ADD FOREIGN KEY (id_fecha) REFERENCES dim_fecha_oec(id_fecha)
''')
mycursor.execute(''' 
ALTER TABLE fact_oec
ADD FOREIGN KEY (id_razon_social) REFERENCES dim_razon_social(id_razon_social)
''')
mycursor.execute(''' 
ALTER TABLE fact_oec
ADD FOREIGN KEY (id_perfil) REFERENCES dim_perfil_oec(id_perfil)
''')


# DATAMART CI

## Tabla dim_curso

In [ ]:
df_curso = pd.read_csv(os.path.join(path_dm_ci,'dim_curso_ci.csv'))
df_curso['id_curso'] = df_curso['id_curso'].astype(str)
df_curso = df_curso.fillna({'costo':0})
df_curso = df_curso.dropna()

In [ ]:
mycursor.execute("DROP TABLE IF EXISTS dim_curso_ci")
mycursor.execute('''CREATE TABLE dim_curso_ci (
id_curso VARCHAR(4) PRIMARY KEY,
curso VARCHAR(255) NOT NULL,
area VARCHAR(255) NOT NULL,
especialidad VARCHAR(255) NOT NULL,
modalidad VARCHAR(63) NOT NULL,
carga_horaria INT NOT NULL,
costo FLOAT NOT NULL
)''')
sql_formula = 'INSERT INTO dim_curso_ci (id_curso, curso, area, especialidad, modalidad, carga_horaria, costo) VALUES (%s,%s,%s,%s,%s,%s,%s)'
for row in tqdm(df_curso.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

## Tabla dim_fecha_ci

In [ ]:
df_fecha_ci = pd.read_csv(os.path.join(path_dm_ci,'dim_fecha_ci.csv'))
df_fecha_ci['id_fecha'] = df_fecha_ci['id_fecha'].astype(str)

In [ ]:
mycursor.execute("DROP TABLE IF EXISTS dim_fecha_ci")
mycursor.execute('''CREATE TABLE dim_fecha_ci (
id_fecha VARCHAR(4) PRIMARY KEY,
anio VARCHAR(15) NOT NULL
)''')
sql_formula = 'INSERT INTO dim_fecha_ci (id_fecha, anio) VALUES (%s,%s)'
for row in tqdm(df_fecha_ci.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

## Tabla fact_ci

In [ ]:
df_fact_ci = fact_ci
df_fact_ci['id_razon_social'] = df_fact_ci['id_razon_social'].astype(str)
df_fact_ci['id_curso'] = df_fact_ci['id_curso'].astype(str)
df_fact_ci['id_fecha'] = df_fact_ci['id_fecha'].astype(str)

In [ ]:
mycursor.execute("DROP TABLE IF EXISTS fact_ci")
mycursor.execute('''CREATE TABLE fact_ci (
id_razon_social VARCHAR(4) NOT NULL,
id_curso VARCHAR(4) NOT NULL,
id_fecha VARCHAR(4) NOT NULL,
num_cap INT NOT NULL,
total_cursos INT NOT NULL,
volumen_capacitados INT NOT NULL,
total_horas INT NOT NULL,
total_ganancias FLOAT NOT NULL
)''')
sql_formula = 'INSERT INTO fact_ci(id_razon_social, id_curso, id_fecha, num_cap, total_cursos, volumen_capacitados, total_horas, total_ganancias) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)'
for row in tqdm(df_fact_ci.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

## CREAR CLAVES FORANEAS CI

In [ ]:
mycursor.execute(''' 
ALTER TABLE fact_ci
ADD FOREIGN KEY (id_fecha) REFERENCES dim_fecha_ci(id_fecha)
''')
mycursor.execute(''' 
ALTER TABLE fact_ci
ADD FOREIGN KEY (id_razon_social) REFERENCES dim_razon_social(id_razon_social)
''')
mycursor.execute(''' 
ALTER TABLE fact_ci
ADD FOREIGN KEY (id_curso) REFERENCES dim_curso_ci(id_curso)
''')